In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

xtest = pd.read_csv('X_test.csv')
xtrain = pd.read_csv('X_train.csv')
ytest = pd.read_csv("y_test.csv")
ytrain = pd.read_csv("y_train.csv")

df = pd.concat([xtest, xtrain], axis=0).sort_values(by='StudentID')
yresult = pd.concat([ytrain, ytest], axis=0).sort_values(by='StudentID')

df = pd.concat([df, yresult.drop(columns='StudentID')], axis=1).drop(columns='StudentID')


In [31]:
#one hot sklearn
from sklearn.preprocessing import OneHotEncoder

for i in df.columns:
    if(df[i].dtype in ['object', 'str']):
        df = pd.concat([pd.get_dummies(df[i], prefix=f'{i}').astype('int64'), df.drop(columns=i)], axis=1)

len(df.columns)
        

59

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

target = np.array(df['G3'])
features = StandardScaler().fit_transform(np.array(df.drop(columns='G3')))


xtrain, xtest, ytrain, ytest = train_test_split(features, target, random_state=23, test_size=0.2)

In [39]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

In [132]:
model = Sequential()

model.add(Dense(64, activation='relu', input_shape=(xtrain.shape[1],)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(64, activation='relu', kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(1))

model.compile(optimizer=Adam(lr=0.02), loss=tf.keras.losses.mae, metrics=[tf.keras.metrics.RootMeanSquaredError])


NameError: name 'rmse' is not defined

In [133]:
train = model.fit(
    xtrain, 
    ytrain,
    epochs=10, 
    batch_size=32,
    validation_data=(xtest, ytest)
)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [127]:
model.predict(xtest)

# accuracy
ypred = model.predict(xtest)

def accuracy(ytest, ypred):
    right = 0
    total = 0
    for i, x in zip(ytest, ypred):
        if(i > x[0]-1 and i < x[0] + 1):
            right += 1
        total += 1
    return right/total

7/7 [==============================] - 0s 1ms/step


In [128]:
accuracy(ytest, ypred)

0.5550239234449761

In [129]:
for i, x in zip(ytest, ypred):
    print(i, round(x[0]))

13 13
12 12
13 12
11 13
11 10
13 14
14 14
14 13
16 16
15 16
15 15
9 11
16 14
8 10
15 16
18 18
9 11
9 11
11 12
11 12
11 11
16 16
16 17
13 14
10 11
14 15
0 10
12 12
12 12
15 15
13 13
13 13
11 12
4 6
13 12
11 11
14 15
14 15
10 11
11 12
7 7
9 9
12 12
0 6
11 11
16 15
13 13
13 12
14 13
11 13
12 13
10 11
14 12
15 13
13 14
14 15
15 16
11 11
16 16
8 7
11 11
13 13
10 11
0 11
11 12
15 15
13 13
15 15
13 12
11 12
10 12
13 13
14 16
11 11
9 11
16 17
12 12
13 13
11 12
11 11
17 16
10 10
14 15
8 9
15 13
9 10
14 12
11 11
10 9
11 12
11 12
13 13
12 12
10 12
10 9
13 13
8 9
13 13
10 10
11 10
16 16
9 11
12 12
10 11
17 15
10 10
18 18
12 12
13 14
13 12
11 12
10 12
13 12
9 12
10 13
13 13
16 15
16 16
12 14
12 13
11 12
13 13
16 16
9 10
10 11
11 12
12 14
10 12
14 13
18 18
17 17
18 17
13 14
10 11
12 13
12 12
11 12
14 14
16 17
10 11
10 11
12 11
10 11
0 7
13 12
0 11
12 11
11 11
10 11
15 13
11 11
11 12
17 18
12 12
8 8
10 13
14 15
18 17
11 12
10 11
6 9
14 15
0 6
10 11
10 11
10 11
12 13
0 1
15 15
6 6
11 12
12 13
15 15
12